<a href="https://colab.research.google.com/github/Mye0n9/AAI_Pitcher/blob/CNN_Testing/Pitcher_CNN_Data_Preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib.patches as patches
import glob
import cv2
import re
import random
import json
import os
from google.colab.patches import cv2_imshow


print("NumPy Version :{}".format(np.__version__))
print("TensorFlow Version :{}".format(tf.__version__))
print("Matplotlib Version :{}".format(plt.matplotlib.__version__))

NumPy Version :1.22.4
TensorFlow Version :2.12.0
Matplotlib Version :3.7.1


# **DATA_PREPROCESSING**

In [2]:
image_data = []
label_data = []
# 9 folders and inside them are 33 folders
for i in range(1,9):
  for j in range(1,33):
    image_link = '/content/drive/MyDrive/Raw_Data/{one}/{two}/*'.format(one = i,two = j)
    label_link = '/content/drive/MyDrive/Label/{one}/{two}/*'.format(one = i,two = j)
    image_data.append(glob.glob(image_link))
    label_data.append(glob.glob(label_link))


In [3]:
# Cropping wrist, elbow and shoulder of the image
# 64 x 64
def cropper (img, point, size):
  # num1 is x
  # num2 is y
  max_x = 1920
  max_y = 1080

  x,y = point[0], point[1]
  x1,x2,y1,y2 = 0,0,0,0

  # x1 is smaller than x2
  # y1 is smaller than y1

  x1 = x-size
  x2 = x+size
  
  y1 = y - size
  y2 = y + size

  cropped_img = img[y1:y2, x1:x2]/255
  # print(cropped_img)
  return cropped_img/255

In [ ]:
IMG_SIZE = 32

# image_data and label_data not compatible
vals_with_error = [96, 136, 146, 197, 220, 223]

for i in range(96,len(image_data)):
  shoulders = []
  elbows = []
  wrists = []
  if i not in vals_with_error:
    for j in range(len(image_data[i])):
      f = open(label_data[i][j])
      img = cv2.imread(image_data[i][j],cv2.IMREAD_COLOR)
      data = json.load(f)
      points = data['annotations'][1]['points']
      # shoulder (5,6 -> 15,16 | 18,19)
      shoulders.append(cropper(img,points[15:17],int(IMG_SIZE/2)))
      shoulders.append(cropper(img,points[18:20],int(IMG_SIZE/2)))
      # elbows (7,8 -> 21,22 | 24,25)
      elbows.append(cropper(img,points[21:23],int(IMG_SIZE/2)))
      elbows.append(cropper(img,points[24:26],int(IMG_SIZE/2)))
      # wrists (9,10 -> 27,28 | 30, 31)
      wrists.append(cropper(img,points[27:29],int(IMG_SIZE/2)))
      wrists.append(cropper(img,points[30:32],int(IMG_SIZE/2)))
      f.close()
    df = pd.DataFrame(zip(wrists, elbows, shoulders), columns = ['wrist', 'elbow','shoulder'])
    df.to_pickle("drive/MyDrive/PKL_PITCHER/img_merged_{num}.pkl".format(num = i))
    # print(i)


# total: 27278 images of wrists, elbows, and shoulders
# takes 1hr 40min
# About to use System RAM of Colab